In [1]:
__author__ = 'Eric Armengaud, Benjamin Weaver <benjamin.weaver@noirlab.edu>, Alice Jacques <alice.jacques@noirlab.edu>'
__version__ = '20230816' # yyyymmdd
__datasets__ = ['sdss_dr16', 'boss_dr16', 'desi_edr']  
__keywords__ = ['sparcl', 'spectroscopy', 'sdss spectra', 'desi spectra', 'tutorial', 'prospect', 'specutils']

# Obtain and plot spectra data using SPARCL, prospect, and specutils

*Credit*: [Eric Armengaud](https://github.com/armengau), Saclay - CEA, is the primary author of prospect. See also the [prospect contributors](https://github.com/desihub/prospect/graphs/contributors).

### Table of contents
* [Goals](#goals)
* [Disclaimer & attribution](#disclaimer)
* [Imports and setup](#imports)
* Find, retrieve, and plot spectra from:
    * [DESI](#desi)
    * [SDSS](#sdss)
    * [(e)BOSS](#boss)

<a class="anchor" id="goals"></a>
## Goals
*Project*: Obtain DESI EDR, SDSS DR16, and (e)BOSS DR16 spectra using the [NOIRLab SPARCL spectrum service](https://astrosparcl.datalab.noirlab.edu), convert data to [specutils objects](https://specutils.readthedocs.io/en/stable/) as needed, and use [prospect](https://desi-prospect.readthedocs.io/en/latest/) to display the data.

*Takeaway*: At no point in this notebook will any data files be opened.

*Takeaway*: prospect allows multiple, independent spectra visualizations to coexist within the same notebook.

This notebook is an adaptation of the [Prospect + specutils + SPARCL](https://github.com/desihub/prospect/blob/main/doc/nb/Prospect_spectrum_service.ipynb) notebook.

<a class="anchor" id="disclaimer"></a>
## Disclaimer & attribution
If you use this notebook for your published science, please acknowledge the following:

* Data Lab concept paper: Fitzpatrick et al., "The NOAO Data Laboratory: a conceptual overview", SPIE, 9149, 2014, http://dx.doi.org/10.1117/12.2057445

* Data Lab disclaimer: https://datalab.noirlab.edu/disclaimers.php

* SPARCL: https://astrosparcl.datalab.noirlab.edu/sparc/acknowledgments/

* prospect: [Eric Armengaud](https://github.com/armengau), Saclay - CEA, the primary author of prospect. See also the [prospect contributors](https://github.com/desihub/prospect/graphs/contributors).

<a class="anchor" id="imports"></a>
## Imports and setup

Note that we are not relying heavily on the DESI software stack (`DESI 23.1`), although some imports will be embedded within `prospect`.

In [2]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import astropy.units as u
from astropy.nddata import InverseVariance
from specutils import __version__ as specutils_version, Spectrum1D
from prospect import __version__ as prospect_version
from prospect.viewer import plotspectra
from prospect.specutils import Spectra
from sparcl import __version__ as sparcl_version
from sparcl.client import SparclClient
from dl import __version__ as dl_version, queryClient as qc
print(f"astro-datalab=={dl_version}")
print(f"specutils=={specutils_version}")
print(f"prospect=={prospect_version}")
print(f"sparcl=={sparcl_version}")

astro-datalab==2.21.3
specutils==1.9.1
prospect==1.2.5
sparcl==1.2.0


### Start SPARCL Client

In [3]:
client = SparclClient()
client

(sparclclient:1.2.0, api:9.0, https://astrosparcl.datalab.noirlab.edu/sparc, verbose=False, connect_timeout=1.1, read_timeout=5400.0)

### Set up Data Lab database interface

All data should be visible in the 'default' profile

In [4]:
qc.get_profile()

'default'

<a class="anchor" id="desi"></a>
## DESI

### Find DESI spectra

SPARCL provides access to DESI spectra that have been coadded by HEALPixel.  This corresponds to entries in the `desi_edr.zpix` table at Astro Data Lab (see column information [here](https://datalab.noirlab.edu/query.php?name=desi_edr.zpix)).

In [5]:
q = """
SELECT z.targetid, z.chi2, z.z, z.zerr, z.zwarn, z.spectype, z.subtype,
       z.coadd_numexp, z.coadd_exptime, z.healpix, z.deltachi2
FROM desi_edr.zpix AS z
WHERE z.zcat_primary
    AND z.survey = 'sv3'
    AND z.program = 'dark'
    AND z.spectype = 'GALAXY'
    AND z.z BETWEEN 0.5 AND 0.9
    AND z.zwarn = 0
ORDER BY z.targetid
LIMIT 50
"""
desi_ids = qc.query(sql=q, fmt='table')
desi_ids

targetid,chi2,z,zerr,zwarn,spectype,subtype,coadd_numexp,coadd_exptime,healpix,deltachi2
int64,float64,float64,float64,int64,str6,int64,int64,float64,int64,float64
1018147809263616,7365.986792743206,0.5242944629168498,6.0633200133966495e-05,0,GALAXY,--,1,733.9571,10517,71.94141682237387
1030495978651648,8103.990616310388,0.7552786543004375,0.00017749836792365145,0,GALAXY,--,1,1228.5404,25934,20.647511329501867
1030507408130048,7649.292701952159,0.6367214865921862,0.00010330595538305078,0,GALAXY,--,1,1521.4265,25599,133.98583871871233
1030526211194880,7539.091522403061,0.8053032876978878,0.00011301271093738162,0,GALAXY,--,1,979.7707,25945,156.96490418165922
1030538257235968,8134.653569459915,0.5719903498779939,3.223043052206872e-05,0,GALAXY,--,2,1719.2445,25957,609.7288934141397
1030538257235969,7135.108882904053,0.5722036123057208,1.620413588830633e-05,0,GALAXY,--,1,979.7707,25957,487.25401762500405
1030550353608705,7867.883817739785,0.7842247194907328,0.0001135860568130856,0,GALAXY,--,3,2420.4485,25968,37.140998892486095
1030568439447553,8961.871326968074,0.754500761465981,0.00010126469934510125,0,GALAXY,--,4,3522.769,25976,10.445683613419533
1030573330006017,8754.344732429832,0.5732047635328416,0.00012256537277712583,0,GALAXY,--,4,5545.4507,27247,310.3479618281126


**QA**: Do we really have 50 unique spectra?

In [6]:
assert (np.unique(desi_ids['targetid']) == desi_ids['targetid']).all()

### Retrieve DESI spectra

With the set of `targetid` obtained above, we can directly retrieve DESI spectra.

In [7]:
include = client.get_all_fields(dataset_list=['DESI-EDR'])
desi_spectra = client.retrieve_by_specid(specid_list=desi_ids['targetid'].value.tolist(),
                                         include=include,
                                         dataset_list=['DESI-EDR'])
desi_spectra.info

{'status': {'success': True,
  'info': ["Successfully found 50 records in dr_list=['DESI-EDR']"],
  'warnings': []}}

**QA**: Did we really find all of the expected spectra?

In [8]:
targetids_sorted = sorted(desi_spectra.records, key=lambda x: x.targetid)
assert (np.array([r.targetid for r in targetids_sorted]) == desi_ids['targetid']).all()
assert all([r.survey == 'sv3' for r in desi_spectra.records])
assert all([r.program == 'dark' for r in desi_spectra.records])

**QA**: Do all spectra records have the same wavelength solution?

In [9]:
assert all([(r.wavelength == desi_spectra.records[0].wavelength).all() for r in desi_spectra.records])

#### View the contents of the first retrieved record

In [10]:
desi_spectra.records[0]

{'specid': 1084094209327112,
 'datasetgroup': 'DESI',
 'redshift_err': 2.58935957851403e-05,
 'telescope': 'kp4m',
 'spectype': 'GALAXY',
 'instrument': 'DESI',
 'targetid': 1084094209327112,
 'wavemax': 9800.0,
 'redshift_warning': 0,
 'dec': 34.4909674837506,
 'ra': 251.046383671818,
 'specprimary': True,
 'exptime': 2043.08386230469,
 'site': 'kpno',
 'redshift': 0.680781688499324,
 'data_release': 'DESI-EDR',
 'wavemin': 3600.0,
 'std_fiber_ra': 0.0,
 'pmra': 0.0,
 'sv3_desi_target': 4611686018427387904,
 'sv3_mws_target': 0,
 'healpix': 9425,
 'fa_type': 1,
 'npixels': 7925,
 'rms_delta_x': 0.004000000189989805,
 'program': 'dark',
 'tsnr2_elg': 169.35247802734375,
 'pmdec': 0.0,
 'deltachi2': 30.95276916027069,
 'priority_init': 1700,
 'target_ra': None,
 'zcat_nspec': 1,
 'cmx_target': 0,
 'sv1_desi_target': 0,
 'mean_mjd': 59325.4455512,
 'objtype': 'TGT',
 'coadd_fiberstatus': 0,
 'sv2_scnd_target': 0,
 'sv_primary': True,
 'plate_dec': 34.49096748375056,
 'plate_ra': 251.0463

### Organize metadata

Prospect needs several inputs:

1. An object containing spectra.  In this case we'll use [`prospect.specutils.Spectra`](https://desi-prospect.readthedocs.io/en/latest/api.html#prospect.specutils.Spectra), which inherits from [`SpectrumList`](https://specutils.readthedocs.io/en/stable/api/specutils.SpectrumList.html#specutils.SpectrumList), and is really just a [`Spectrum1D`](https://specutils.readthedocs.io/en/stable/api/specutils.Spectrum1D.html#specutils.Spectrum1D) object underneath.
   * The object contains the usual flux, wavelength, uncertainty.
   * In addtion a "fibermap" table is needed. This should be an Astropy `Table` with the expected columns.
2. A redshift catalog. This should be an Astropy `Table` with the expected columns.
3. A model spectrum.  The model is actually provided by SPARCL, but we need to input it separately.

#### Spectrum object

First we assemble the components of the spectrum object

In [11]:
flux = np.zeros((len(desi_spectra.records), desi_spectra.records[0].flux.shape[0]),
                dtype=desi_spectra.records[0].flux.dtype)
uncertainty = np.zeros((len(desi_spectra.records), desi_spectra.records[0].ivar.shape[0]),
                       dtype=desi_spectra.records[0].ivar.dtype)
mask = np.zeros((len(desi_spectra.records), desi_spectra.records[0].mask.shape[0]),
                dtype=desi_spectra.records[0].mask.dtype)

meta = {'sparcl_id': list(), 'data_release': list()}
sparcl_id = list()
data_release = list()

for k in range(len(desi_spectra.records)):
    flux[k, :] = desi_spectra.records[k].flux
    uncertainty[k, :] = desi_spectra.records[k].ivar
    mask[k, :] = desi_spectra.records[k].mask
    meta['sparcl_id'].append(desi_spectra.records[k].sparcl_id)
    meta['data_release'].append(desi_spectra.records[k].data_release)

And the "fibermap" table. We'll start with photometric quantities.

In [12]:
columns = ('targetid', 'ra', 'dec', 'ref_epoch', 'pmra', 'pmdec', 'ebv',
           'flux_g', 'flux_r', 'flux_z', 'flux_w1', 'flux_w2')
q = """
SELECT {0}
FROM desi_edr.photometry
WHERE targetid IN ({1})
ORDER BY targetid
""".format(', '.join(columns), ', '.join([str(t) for t in desi_ids['targetid'].value.tolist()]))

fibermap = qc.query(sql=q, fmt='table')

In [13]:
for col in fibermap.colnames:
    if col == 'ra' or col == 'dec':
        fibermap.rename_column(col, 'TARGET_' + col.upper())
    else:
        fibermap.rename_column(col, col.upper())

**QA**: Did we find photometry for every `targetid`?

In [14]:
assert (fibermap['TARGETID'] == desi_ids['targetid']).all()

Next we add targeting bitmasks.

In [15]:
columns = ('targetid', 'sv3_desi_target', 'sv3_bgs_target', 'sv3_mws_target', 'sv3_scnd_target')
q = """
SELECT DISTINCT {0}
FROM desi_edr.target
WHERE targetid IN ({1})
ORDER BY targetid
""".format(', '.join(columns), ', '.join([str(t) for t in fibermap['TARGETID'].value.tolist()]))

targeting = qc.query(sql=q, fmt='table')

In [16]:
for col in targeting.colnames:
    targeting.rename_column(col, col.upper())

**QA**: Did we find targeting for every `targetid`?

In [17]:
assert (targeting['TARGETID'] == fibermap['TARGETID']).all()

Add columns into `fibermap`.

In [18]:
for col in ('SV3_DESI_TARGET', 'SV3_BGS_TARGET', 'SV3_MWS_TARGET', 'SV3_SCND_TARGET'):
    fibermap.add_column(targeting[col])
fibermap

TARGETID,TARGET_RA,TARGET_DEC,REF_EPOCH,PMRA,PMDEC,EBV,FLUX_G,FLUX_R,FLUX_Z,FLUX_W1,FLUX_W2,SV3_DESI_TARGET,SV3_BGS_TARGET,SV3_MWS_TARGET,SV3_SCND_TARGET
int64,float64,float64,float64,int64,int64,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64
1018147809263616,218.15530210149535,35.769959220507396,0.0,0,0,0.011083154,0.48597622,1.2058442,1.5106901,2.482841,1.3636942,4611686018427387904,0,0,2147483648
1030495978651648,218.82868695999053,-1.4239743152821387,0.0,0,0,0.040679332,0.6364655,1.2522537,4.010964,13.028034,7.1272125,4611686018427387904,0,0,17179869184
1030507408130048,180.1889661147553,-0.9613644465384479,0.0,0,0,0.024523733,0.48548192,1.483542,4.9795527,15.609289,7.899148,4611686018427387904,0,0,17179869184
1030526211194880,220.97916752669602,-0.12526168095554563,0.0,0,0,0.03961234,2.365766,4.175312,9.775242,57.561703,120.945496,4611686018427387904,0,0,17179869184
1030538257235968,218.87134476858458,0.3034533715013907,0.0,0,0,0.038472127,1.699776,4.9637136,11.5170145,23.5799,13.483782,4611686018427387904,0,0,17179869184
1030538257235969,218.8711,0.3036,2015.5,0,0,0.0,0.0,0.0,0.0,0.0,0.0,4611686018427387904,0,0,17179869184
1030550353608705,219.75550234102283,0.8547659846230762,0.0,0,0,0.04603932,0.748148,1.0393113,2.4640872,7.8900027,14.943021,4611686018427387904,0,0,17179869184
1030568439447553,217.962,1.5059,2015.5,0,0,0.0,0.0,0.0,0.0,0.0,0.0,4611686018427387904,0,0,17179869184
1030573330006017,149.459078510057,1.7517805116389942,0.0,0,0,0.021018779,0.29732797,0.36820117,0.07077911,0.44191432,5.732679,4611686018427387904,0,0,17179869184


Finally assemble the object.

In [19]:
desi_prospect = Spectra(bands=['coadd'],
                        wave={'coadd': desi_spectra.records[0].wavelength},
                        flux={'coadd': flux},
                        ivar={'coadd': uncertainty},
                        mask={'coadd': mask},
                        fibermap=fibermap,
                        meta={'coadd': meta})

#### Redshift catalog

We can re-use the initial query above; it was deliberately constructed.

In [20]:
desi_zcatalog = desi_ids.copy()
for col in desi_zcatalog.colnames:
    if col == 'healpix':
        desi_zcatalog.rename_column(col, 'HPXPIXEL')
    else:
        desi_zcatalog.rename_column(col, col.upper())
desi_zcatalog

TARGETID,CHI2,Z,ZERR,ZWARN,SPECTYPE,SUBTYPE,COADD_NUMEXP,COADD_EXPTIME,HPXPIXEL,DELTACHI2
int64,float64,float64,float64,int64,str6,int64,int64,float64,int64,float64
1018147809263616,7365.986792743206,0.5242944629168498,6.0633200133966495e-05,0,GALAXY,--,1,733.9571,10517,71.94141682237387
1030495978651648,8103.990616310388,0.7552786543004375,0.00017749836792365145,0,GALAXY,--,1,1228.5404,25934,20.647511329501867
1030507408130048,7649.292701952159,0.6367214865921862,0.00010330595538305078,0,GALAXY,--,1,1521.4265,25599,133.98583871871233
1030526211194880,7539.091522403061,0.8053032876978878,0.00011301271093738162,0,GALAXY,--,1,979.7707,25945,156.96490418165922
1030538257235968,8134.653569459915,0.5719903498779939,3.223043052206872e-05,0,GALAXY,--,2,1719.2445,25957,609.7288934141397
1030538257235969,7135.108882904053,0.5722036123057208,1.620413588830633e-05,0,GALAXY,--,1,979.7707,25957,487.25401762500405
1030550353608705,7867.883817739785,0.7842247194907328,0.0001135860568130856,0,GALAXY,--,3,2420.4485,25968,37.140998892486095
1030568439447553,8961.871326968074,0.754500761465981,0.00010126469934510125,0,GALAXY,--,4,3522.769,25976,10.445683613419533
1030573330006017,8754.344732429832,0.5732047635328416,0.00012256537277712583,0,GALAXY,--,4,5545.4507,27247,310.3479618281126


#### Model spectra

Prospect expects a model in the form of a tuple containing wavelength and flux. Since SPARCL provides the model, this is easy.  There are other ways to specify the model, but these require more access to the DESI software stack *and* data *files*.

In [21]:
model_flux = np.zeros((len(desi_spectra.records), desi_spectra.records[0].model.shape[0]),
                      dtype=desi_spectra.records[0].model.dtype)

for k in range(len(desi_spectra.records)):
        model_flux[k, :] = desi_spectra.records[k].model
        
desi_model = (desi_spectra.records[0].wavelength, model_flux)

### Start prospect

With everything assembled, the interface to prospect is just a single call.

In [ ]:
plotspectra(desi_prospect,
            zcatalog=desi_zcatalog,
            redrock_cat=None,
            notebook=True,
            with_thumb_tab=False,
            with_vi_widgets=False,
            with_coaddcam=False,
            mask_type='SV3_DESI_TARGET',
            model_from_zcat=False,
            model=desi_model)

<a class="anchor" id="sdss"></a>
## SDSS

### Find SDSS spectra

In [23]:
q = """
SELECT z.specobjid, z.bestobjid, z.z, z.zerr, z.zwarning, z.class, z.subclass,
       z.rchi2diff, z.primtarget, z.sectarget
FROM sdss_dr16.specobj AS z
WHERE z.bestobjid > 0
    AND z.run2d = '26'
    AND z.plate = 2955
    AND z.mjd = 54562
    AND z.class = 'GALAXY'
    AND z.zwarning = 0
ORDER BY z.specobjid
LIMIT 50
"""
sdss_ids = qc.query(sql=q, fmt='table')
sdss_ids

specobjid,bestobjid,z,zerr,zwarning,class,subclass,rchi2diff,primtarget,sectarget
int64,int64,float64,float64,int64,str6,str11,float64,int64,int64
3327035125891360768,1237655468065620461,0.26219922,6.426468e-05,0,GALAXY,--,0.16153336,96,0
3327036500280895488,1237655468065489538,0.21097003,5.7919853e-05,0,GALAXY,--,0.13921309,64,0
3327038424426244096,1237655468065620456,0.2605426,6.229882e-05,0,GALAXY,--,0.22558784,32,0
3327038974182057984,1237655468065620382,0.11276812,1.5994665e-05,0,GALAXY,--,0.510905,64,0
3327040073693685760,1237651736318574966,0.06364931,2.8312488e-05,0,GALAXY,--,0.42436635,64,0
3327040348571592704,1237651736318640378,0.20836118,4.189276e-05,0,GALAXY,--,1.1794078,96,0
3327040898327406592,1237655468602491146,0.20784536,4.1236628e-05,0,GALAXY,--,0.61972916,64,0
3327041722961127424,1237651736318575080,0.1408417,1.9260546e-05,0,GALAXY,STARFORMING,0.49670327,64,0
3327042822472755200,1237651736318574838,0.21259658,7.138862e-05,0,GALAXY,--,0.14038157,96,0


**QA**: Do we really have 50 unique spectra?

In [24]:
assert (np.unique(sdss_ids['specobjid']) == sdss_ids['specobjid']).all()

### Retrieve SDSS spectra

With the set of `specobjid` obtained above, we can directly retrieve SDSS spectra.

In [25]:
include = client.get_all_fields(dataset_list=['SDSS-DR16'])
sdss_spectra = client.retrieve_by_specid(specid_list=sdss_ids['specobjid'].value.tolist(),
                                         include=include,
                                         dataset_list=['SDSS-DR16'])
sdss_spectra.info

{'status': {'success': True,
  'info': ["Successfully found 50 records in dr_list=['SDSS-DR16']"],
  'warnings': []}}

**QA**: Did we really find all of the expected spectra? Note that we have to assume `specid == specobjid`.

In [26]:
sdss_specids_sorted = sorted(sdss_spectra.records, key=lambda x: x.specid)
assert (np.array([r.specid for r in sdss_specids_sorted]) == sdss_ids['specobjid']).all()
assert all([r.plate == 2955 for r in sdss_spectra.records])
assert all([r.mjd == 54562 for r in sdss_spectra.records])

**QA**: Do all spectra records have the same wavelength solution?

In [27]:
assert all([(r.wavelength == sdss_spectra.records[0].wavelength).all() for r in sdss_spectra.records])

#### View the contents of the first retrieved record

In [28]:
sdss_spectra.records[0]

{'specid': 3327046121007638528,
 'datasetgroup': 'SDSS_BOSS',
 'redshift_err': 2.27969903789926e-05,
 'telescope': 'sloan25m',
 'spectype': 'GALAXY',
 'instrument': 'SDSS',
 'targetid': 1237655468065292360,
 'wavemax': 9191.7880859375,
 'redshift_warning': 0,
 'dec': 0.79150642,
 'ra': 235.80015,
 'specprimary': True,
 'exptime': 3002.0,
 'site': 'apo',
 'redshift': 0.0843528062105179,
 'data_release': 'SDSS-DR16',
 'wavemin': 3812.41357421875,
 'fiberid': 43,
 'class_noqso': '',
 'special_target1': 0,
 'elodie_filename': '',
 'platesn2': 20.60099983215332,
 'targetobjid': '     11268994536964194',
 'specsegue': 0,
 'calibflux': [5.839175701141357,
  32.200157165527344,
  82.3163070678711,
  126.0885009765625,
  179.93894958496094],
 'subclass_noqso': '',
 'spec2_i': 23.063199996948242,
 'rchi2': 0.9485337138175964,
 'survey': 'sdss',
 'segue1_target1': 0,
 'plate': 2955,
 'elodie_object': '',
 'mjd': 54562,
 'vdispz': 0.0,
 'spec1_i': 20.87619972229004,
 'eboss_target2': 0,
 'elodie_f

### Organize metadata

Prospect needs several inputs:

1. An object containing spectra.  In this case we'll use a [`Spectrum1D`](https://specutils.readthedocs.io/en/stable/api/specutils.Spectrum1D.html#specutils.Spectrum1D) object.
   * The object contains the usual flux, wavelength, uncertainty.
   * In addtion a "plugmap" table is needed. This should be an Astropy `Table` with the expected columns.
2. A redshift catalog. This should be an Astropy `Table` with the expected columns.
3. A model spectrum.  The model is actually provided by SPARCL, but we need to input it separately.

#### Spectrum object

First we assemble the components of the spectrum object

In [29]:
flux = np.zeros((len(sdss_spectra.records), sdss_spectra.records[0].flux.shape[0]),
                dtype=sdss_spectra.records[0].flux.dtype)
uncertainty = np.zeros((len(sdss_spectra.records), sdss_spectra.records[0].ivar.shape[0]),
                       dtype=sdss_spectra.records[0].ivar.dtype)
mask = np.zeros((len(sdss_spectra.records), sdss_spectra.records[0].mask.shape[0]),
                dtype=sdss_spectra.records[0].mask.dtype)

meta = {'sparcl_id': list(), 'data_release': list()}
sparcl_id = list()
data_release = list()

for k in range(len(sdss_spectra.records)):
    flux[k, :] = sdss_spectra.records[k].flux
    uncertainty[k, :] = sdss_spectra.records[k].ivar
    mask[k, :] = sdss_spectra.records[k].mask
    meta['sparcl_id'].append(sdss_spectra.records[k].sparcl_id)
    meta['data_release'].append(sdss_spectra.records[k].data_release)

And the "plugmap" table. We'll start with photometric quantities.

In [30]:
columns = ('objid', 'ra', 'dec', 'u', 'g', 'r', 'i', 'z')
q = """
SELECT {0}
FROM sdss_dr16.photoplate
WHERE objid IN ({1})
ORDER BY objid
""".format(', '.join(columns), ', '.join([str(t) for t in sdss_ids['bestobjid'].value.tolist()]))

plugmap = qc.query(sql=q, fmt='table')

In [31]:
for col in plugmap.colnames:
    plugmap.rename_column(col, col.upper())
mag = np.zeros((len(plugmap), 5), dtype=plugmap['G'].value.dtype)
for k, band in enumerate('UGRIZ'):
    mag[:, k] = plugmap[band].value
plugmap.add_column(mag, name='MAG')
plugmap.add_column(sdss_ids['primtarget'], name='PRIMTARGET')
plugmap.add_column(sdss_ids['sectarget'], name='SECTARGET')
meta['plugmap'] = plugmap

**QA**: Did we find photometry for every `bestobjid`?

In [32]:
assert (plugmap['OBJID'] == sorted(sdss_ids['bestobjid'])).all()

Finally assemble the object.

In [33]:
sdss_prospect = Spectrum1D(flux=flux * u.Unit('1e-17 erg / (Angstrom cm2 s)'),
                           spectral_axis=sdss_spectra.records[0].wavelength * u.Unit('Angstrom'),
                           uncertainty=InverseVariance(uncertainty),
                           mask=mask != 0,
                           meta=meta)

#### Redshift catalog

We can re-use the initial query above; it was deliberately constructed.

In [34]:
sdss_zcatalog = sdss_ids.copy()
for col in sdss_zcatalog.colnames:
    if col == 'zerr':
        sdss_zcatalog.rename_column(col, 'Z_ERR')
    else:
        sdss_zcatalog.rename_column(col, col.upper())
sdss_zcatalog

SPECOBJID,BESTOBJID,Z,Z_ERR,ZWARNING,CLASS,SUBCLASS,RCHI2DIFF,PRIMTARGET,SECTARGET
int64,int64,float64,float64,int64,str6,str11,float64,int64,int64
3327035125891360768,1237655468065620461,0.26219922,6.426468e-05,0,GALAXY,--,0.16153336,96,0
3327036500280895488,1237655468065489538,0.21097003,5.7919853e-05,0,GALAXY,--,0.13921309,64,0
3327038424426244096,1237655468065620456,0.2605426,6.229882e-05,0,GALAXY,--,0.22558784,32,0
3327038974182057984,1237655468065620382,0.11276812,1.5994665e-05,0,GALAXY,--,0.510905,64,0
3327040073693685760,1237651736318574966,0.06364931,2.8312488e-05,0,GALAXY,--,0.42436635,64,0
3327040348571592704,1237651736318640378,0.20836118,4.189276e-05,0,GALAXY,--,1.1794078,96,0
3327040898327406592,1237655468602491146,0.20784536,4.1236628e-05,0,GALAXY,--,0.61972916,64,0
3327041722961127424,1237651736318575080,0.1408417,1.9260546e-05,0,GALAXY,STARFORMING,0.49670327,64,0
3327042822472755200,1237651736318574838,0.21259658,7.138862e-05,0,GALAXY,--,0.14038157,96,0


#### Model spectra

Prospect expects a model in the form of a tuple containing wavelength and flux. Since SPARCL provides the model, this is easy.  There are other ways to specify the model, but these require more access to the SDSS *files*.

In [35]:
model_flux = np.zeros((len(sdss_spectra.records), sdss_spectra.records[0].model.shape[0]),
                      dtype=sdss_spectra.records[0].model.dtype)

for k in range(len(sdss_spectra.records)):
        model_flux[k, :] = sdss_spectra.records[k].model
        
sdss_model = (sdss_spectra.records[0].wavelength, model_flux)

### Start prospect

With everything assembled, the interface to prospect is just a single call.

In [ ]:
plotspectra(sdss_prospect,
            zcatalog=sdss_zcatalog,
            redrock_cat=None,
            notebook=True,
            with_thumb_tab=False,
            with_vi_widgets=False,
            with_coaddcam=False,
            mask_type='PRIMTARGET',
            model_from_zcat=False,
            model=sdss_model)

<a class="anchor" id="boss"></a>
## (e)BOSS

### Find (e)BOSS spectra

In [37]:
q = """
SELECT z.specobjid, z.bestobjid, z.z, z.zerr, z.zwarning, z.class, z.subclass,
       z.rchi2diff, z.boss_target1, z.eboss_target0, z.eboss_target1,
       z.eboss_target2
FROM sdss_dr16.specobj AS z
WHERE z.bestobjid > 0
    AND z.run2d = 'v5_13_0'
    AND z.plate = 9599
    AND z.mjd = 58131
    AND z.class = 'GALAXY'
    AND z.zwarning = 0
ORDER BY z.specobjid 
LIMIT 50
"""
boss_ids = qc.query(sql=q, fmt='table')
boss_ids

specobjid,bestobjid,z,zerr,zwarning,class,subclass,rchi2diff,boss_target1,eboss_target0,eboss_target1,eboss_target2
int64,int64,float64,float64,int64,str6,int64,float64,int64,int64,int64,int64
-7639230456632422400,1237665127987282622,0.88640094,7.4862786e-05,0,GALAXY,--,0.06782746,0,0,17592186044416,562949953421312
-7639227982731259904,1237665098459972441,0.91102755,0.00010790457,0,GALAXY,--,0.017501831,0,0,17592186044416,562949953421312
-7639227707853352960,1237665128524284395,0.38422388,3.725687e-05,0,GALAXY,--,0.04291892,0,0,17592186044416,562949953421312
-7639227432975446016,1237665098459972179,0.80037284,7.310519e-05,0,GALAXY,--,0.027097106,0,0,17592186044416,562949953421312
-7639226333463818240,1237665128524350020,0.7552472,0.00021918201,0,GALAXY,--,0.013135433,0,0,17592186044416,562949953421312
-7639225783708004352,1237665128524415517,0.821098,8.6746266e-05,0,GALAXY,--,0.024078012,0,0,17592186044416,562949953421312
-7639225233952190464,1237665098460037767,1.4448227,0.28259814,0,GALAXY,--,0.011102319,0,0,17592186044416,562949953421312
-7639224959074283520,1237665128524153288,0.921071,0.00011867128,0,GALAXY,--,0.018088818,0,0,17592186044416,562949953421312
-7639224409318469632,1237665097923035567,0.9825296,0.00013494636,0,GALAXY,--,0.012501717,0,0,17592186044416,562949953421312


**QA**: Do we really have 50 unique spectra?

In [38]:
assert (np.unique(boss_ids['specobjid']) == boss_ids['specobjid']).all()

### Retrieve (e)BOSS spectra

With the set of `specobjid` obtained above, we can directly retrieve (e)BOSS spectra.

In [39]:
include = client.get_all_fields(dataset_list=['BOSS-DR16'])
boss_spectra = client.retrieve_by_specid(specid_list=boss_ids['specobjid'].value.tolist(),
                                         include=include,
                                         dataset_list=['BOSS-DR16'])
boss_spectra.info

{'status': {'success': True,
  'info': ["Successfully found 50 records in dr_list=['BOSS-DR16']"],
  'warnings': []}}

**QA**: Did we really find all of the expected spectra?

In [40]:
boss_specids_sorted = sorted(boss_spectra.records, key=lambda x: x.specid)
assert (np.array([r.specid for r in boss_specids_sorted]) == boss_ids['specobjid']).all()
assert all([r.plate == 9599 for r in boss_spectra.records])
assert all([r.mjd == 58131 for r in boss_spectra.records])

**QA**: Do all spectra records have the same wavelength solution?

In [41]:
assert all([(r.wavelength == boss_spectra.records[0].wavelength).all() for r in boss_spectra.records])

#### View the contents of the first retrieved record

In [42]:
boss_spectra.records[0]

{'specid': -7639227982731259904,
 'datasetgroup': 'SDSS_BOSS',
 'redshift_err': 0.000107904568722006,
 'telescope': 'sloan25m',
 'spectype': 'GALAXY',
 'instrument': 'BOSS',
 'targetid': 1237665098459972441,
 'wavemax': 10332.37109375,
 'redshift_warning': 0,
 'dec': 24.150201,
 'ra': 134.23291,
 'specprimary': True,
 'exptime': 3600.35,
 'site': 'apo',
 'redshift': 0.911027550697327,
 'data_release': 'BOSS-DR16',
 'wavemin': 3601.63745117188,
 'fiberid': 10,
 'class_noqso': 'GALAXY',
 'special_target1': 0,
 'elodie_filename': '',
 'platesn2': 5.494349956512451,
 'targetobjid': '',
 'specsegue': 0,
 'calibflux': [-0.2695178985595703,
  0.4001877009868622,
  1.0699976682662964,
  1.9848341941833496,
  1.4778335094451904],
 'subclass_noqso': '',
 'spec2_i': 16.17180061340332,
 'rchi2': 1.056380033493042,
 'survey': 'eboss',
 'segue1_target1': 0,
 'plate': 9599,
 'elodie_object': '',
 'mjd': 58131,
 'vdispz': 0.0,
 'spec1_i': 15.260899543762207,
 'eboss_target2': 562949953421312,
 'elodie

### Organize metadata

Prospect needs several inputs:

1. An object containing spectra.  In this case we'll use a [`Spectrum1D`](https://specutils.readthedocs.io/en/stable/api/specutils.Spectrum1D.html#specutils.Spectrum1D) object.
   * The object contains the usual flux, wavelength, uncertainty.
   * In addtion a "plugmap" table is needed. This should be an Astropy `Table` with the expected columns.
2. A redshift catalog. This should be an Astropy `Table` with the expected columns.
3. A model spectrum.  The model is actually provided by SPARCL, but we need to input it separately.

#### Spectrum object

First we assemble the components of the spectrum object

In [43]:
flux = np.zeros((len(boss_spectra.records), boss_spectra.records[0].flux.shape[0]),
                dtype=boss_spectra.records[0].flux.dtype)
uncertainty = np.zeros((len(boss_spectra.records), boss_spectra.records[0].ivar.shape[0]),
                       dtype=boss_spectra.records[0].ivar.dtype)
mask = np.zeros((len(boss_spectra.records), boss_spectra.records[0].mask.shape[0]),
                dtype=boss_spectra.records[0].mask.dtype)

meta = {'sparcl_id': list(), 'data_release': list()}
sparcl_id = list()
data_release = list()

for k in range(len(boss_spectra.records)):
    flux[k, :] = boss_spectra.records[k].flux
    uncertainty[k, :] = boss_spectra.records[k].ivar
    mask[k, :] = boss_spectra.records[k].mask
    meta['sparcl_id'].append(boss_spectra.records[k].sparcl_id)
    meta['data_release'].append(boss_spectra.records[k].data_release)

And the "plugmap" table. We'll start with photometric quantities.

In [44]:
columns = ('objid', 'ra', 'dec', 'u', 'g', 'r', 'i', 'z')
q = """
SELECT {0}
FROM sdss_dr16.photoplate
WHERE objid IN ({1})
ORDER BY objid
""".format(', '.join(columns), ', '.join([str(t) for t in boss_ids['bestobjid'].value.tolist()]))

plugmap = qc.query(sql=q, fmt='table')

In [45]:
for col in plugmap.colnames:
    plugmap.rename_column(col, col.upper())
mag = np.zeros((len(plugmap), 5), dtype=plugmap['G'].value.dtype)
for k, band in enumerate('UGRIZ'):
    mag[:, k] = plugmap[band].value
plugmap.add_column(mag, name='MAG')
plugmap.add_column(boss_ids['boss_target1'], name='BOSS_TARGET1')
plugmap.add_column(boss_ids['eboss_target0'], name='EBOSS_TARGET0')
plugmap.add_column(boss_ids['eboss_target1'], name='EBOSS_TARGET1')
plugmap.add_column(boss_ids['eboss_target2'], name='EBOSS_TARGET2')
meta['plugmap'] = plugmap

**QA**: Did we find photometry for every `bestobjid`?

In [46]:
assert (plugmap['OBJID'] == sorted(boss_ids['bestobjid'])).all()

Finally assemble the object.

In [47]:
boss_prospect = Spectrum1D(flux=flux * u.Unit('1e-17 erg / (Angstrom cm2 s)'),
                           spectral_axis=boss_spectra.records[0].wavelength * u.Unit('Angstrom'),
                           uncertainty=InverseVariance(uncertainty),
                           mask=mask != 0,
                           meta=meta)

#### Redshift catalog

We can re-use the initial query above; it was deliberately constructed.

In [48]:
boss_zcatalog = boss_ids.copy()
for col in boss_zcatalog.colnames:
    if col == 'zerr':
        boss_zcatalog.rename_column(col, 'Z_ERR')
    else:
        boss_zcatalog.rename_column(col, col.upper())
boss_zcatalog

SPECOBJID,BESTOBJID,Z,Z_ERR,ZWARNING,CLASS,SUBCLASS,RCHI2DIFF,BOSS_TARGET1,EBOSS_TARGET0,EBOSS_TARGET1,EBOSS_TARGET2
int64,int64,float64,float64,int64,str6,int64,float64,int64,int64,int64,int64
-7639230456632422400,1237665127987282622,0.88640094,7.4862786e-05,0,GALAXY,--,0.06782746,0,0,17592186044416,562949953421312
-7639227982731259904,1237665098459972441,0.91102755,0.00010790457,0,GALAXY,--,0.017501831,0,0,17592186044416,562949953421312
-7639227707853352960,1237665128524284395,0.38422388,3.725687e-05,0,GALAXY,--,0.04291892,0,0,17592186044416,562949953421312
-7639227432975446016,1237665098459972179,0.80037284,7.310519e-05,0,GALAXY,--,0.027097106,0,0,17592186044416,562949953421312
-7639226333463818240,1237665128524350020,0.7552472,0.00021918201,0,GALAXY,--,0.013135433,0,0,17592186044416,562949953421312
-7639225783708004352,1237665128524415517,0.821098,8.6746266e-05,0,GALAXY,--,0.024078012,0,0,17592186044416,562949953421312
-7639225233952190464,1237665098460037767,1.4448227,0.28259814,0,GALAXY,--,0.011102319,0,0,17592186044416,562949953421312
-7639224959074283520,1237665128524153288,0.921071,0.00011867128,0,GALAXY,--,0.018088818,0,0,17592186044416,562949953421312
-7639224409318469632,1237665097923035567,0.9825296,0.00013494636,0,GALAXY,--,0.012501717,0,0,17592186044416,562949953421312


#### Model spectra

Prospect expects a model in the form of a tuple containing wavelength and flux. Since SPARCL provides the model, this is easy.  There are other ways to specify the model, but these require more access to the (e)BOSS *files*.

In [49]:
model_flux = np.zeros((len(boss_spectra.records), boss_spectra.records[0].model.shape[0]),
                      dtype=boss_spectra.records[0].model.dtype)

for k in range(len(boss_spectra.records)):
        model_flux[k, :] = boss_spectra.records[k].model
        
boss_model = (boss_spectra.records[0].wavelength, model_flux)

### Start prospect

With everything assembled, the interface to prospect is just a single call.

In [ ]:
plotspectra(boss_prospect,
            zcatalog=boss_zcatalog,
            redrock_cat=None,
            notebook=True,
            with_thumb_tab=False,
            with_vi_widgets=False,
            with_coaddcam=False,
            mask_type='EBOSS_TARGET1',
            model_from_zcat=False,
            model=boss_model)